In [ ]:
import os
import json 
import time

import numpy as np
import torch

import matplotlib.pyplot as plt

from sympy import lambdify
import sympy as sp

from pysr import PySRRegressor

import nesymres
from nesymres.architectures.model import Model
from nesymres.utils import load_metadata_hdf5
from nesymres.dclasses import FitParams, NNEquation, BFGSParams
import omegaconf

from pathlib import Path
from functools import partial

In [ ]:
# parameters for running benchmarks

distribution_type = "uniform"
# for normal distribution, use mean and standard deviation.
# for uniform distribution the range is the min and max values
distribution_range = [-.0, 4.0]

number_points = 256
number_trials = 1 # seeds will be trial number
logging = True


In [ ]:
temp_0 = lambda x: x**5. + x**4. + x**3+x**2 +x**1
temp_1 = lambda x: x**4. + x**3+x**2 +x**1

import numpy as np

x = np.random.rand(100,1) * 2 - 1.0

np.mean((temp_0(x) - temp_1(x))**2)

In [ ]:
complex_1 = "1.0*x0/(1.0*x0*1.0*exp(1.0*x0)*1.0*exp(1.0*sin(1.0*x0+1.0))+1.0*exp(1.0*x0)*1.0*exp(1.0*sin(1.0*x0+1.0)))+1.0"
complex_2 = "1.0*sqrt(1.0*abs(1.0*sqrt(1.0*abs(1.0*x0+1.0))))*1.0*sin(1.0*x0/(1.0*x0+1.0)+1.0/(1.0*x0+1.0))+1.0"
complex_3 = "1.0*sqrt(1.0*abs(1.0*x0/(1.0*x0+1.0)+1.0/(1.0*x0+1.0)))+1.0*sqrt(1.0*abs(1.0*exp(1.0*x0)))+1.0"
complex_4 = "1.0*sqrt(1.0*abs(1.0*x0))+1.0/(x0*1.0*sqrt(1.0*abs(1.0*x0))*1.0*sqrt(1.0*abs(1.0*x0)))+1.0"
complex_5 = "1.0*x0/(1.0*x0*1.0*sqrt(1.0*abs(1.0*log(1.0*x0)))+1.0*sqrt(1.0*abs(1.0*log(1.0*x0))))+1.0"
complex_6 = "1.0*x0**2*1.0*sqrt(1.0*abs(1.0*x0+1.0))+1.0*x0+1.0*x0/(1.0*x0)+1.0*log(1.0*x0+1.0)+1.0"
complex_7 = "1.0*sqrt(1.0*abs(1.0*x0*1.0*sqrt(1.0*abs(1.0*x0))/(1.0*x0+1.0)+1.0/(1.0*x0+1.0)))+1.0"
complex_8 = "1.0*x0**2+1.0*x0+1.0*exp(1.0*x0)/1.0*sqrt(1.0*abs(1.0*sqrt(1.0*abs(1.0*x0+1.0))))+1.0"

sample_meta = {complex_1: (-3, 3., 20),
               complex_2: (-3, 3., 20),
               complex_3: (-3, 3., 20),
               complex_4: (0.1, 4., 20),
               complex_5: (1.01, 4, 20),
               complex_6: (-.9, 3., 20),
               complex_7: (-3, 3., 20),
               complex_8: (-3, 3., 20)
              }

benchmark_eqns = [complex_1, complex_2, complex_3, complex_4, \
        complex_5, complex_6, complex_7, complex_8]

In [ ]:
# visualize equations

plt.figure()
for number, fn in enumerate(benchmark_eqns):

    my_fn = sp.lambdify("x0", expr=fn)
    
    (bottom, top, c) = sample_meta[fn]
    
    step_size = (top-bottom)/10000
    x = np.arange(bottom, top, step_size)
    
    plt.plot(x, my_fn(x), label = f"complex-{1+number}")
    print(x.shape, my_fn(x).shape)
    print(f" {1+number} {sp.simplify(fn)} \n")
    print(f"  {sp.expand(fn)} \n")
    print(fn)
    
plt.legend()
plt.title("complex benchmark equations")
plt.show()

In [ ]:

columns = "eqn, seed, pred, target, correct, mse, method, range_low, range_high, number_points"
my_method = "PySR"
eval_tag = f"eval_complex_{my_method}_{int(time.time())}"

if logging:
    with open(f"{eval_tag}.csv", "w") as f:
        f.write(columns)
        f.write("\n")
    
accuracies = []
all_mses = []
all_mse_sds = []
for hh, eqn in enumerate(benchmark_eqns):
    
    equivalents = []
    mses = []
    for trial in range(number_trials):

        np.random.seed(trial)
        torch.manual_seed(trial)
        
        my_fn = sp.lambdify("x0", expr=eqn)
        
        (bottom, top, number_samples) = sample_meta[eqn]
        x = np.random.rand(number_samples, 1) \
                * (top-bottom) \
                + bottom
        y = my_fn(x)
        
        model = PySRRegressor(
            niterations=10,
            binary_operators=["+", "*"],
            unary_operators=[
                "cos",
                "exp",
                "sin",
                "inv(x) = 1/x"  # Custom operator (julia syntax)
            ],
            model_selection="best",
            deterministic = True,
            procs = 0,
            multithreading = False,
            random_state = trial,
            verbosity=0,
            loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
        )
        
        model.fit(x, y)
        
        if "bs" in model.get_best()["equation"]:
            print("abs detected")
        
        if (x >= 0).all():
            # remove abs() if input only consists of positive numbers
            if "abs" in model.get_best()["equation"]:
                best_eqn = sp.simplify(model.get_best()["equation"]\
                                       .replace("inv(", "1./(").replace("abs(x", "(x"))
                best_fn = sp.lambdify("x0", expr=model.get_best()["equation"]\
                                      .replace("inv(", "1./(").replace("abs(x", "(x"))
                
                tgt_fn = sp.lambdify("x0", expr=eqn)
            else:
                best_eqn = sp.simplify(model.get_best()["equation"].replace("inv(", "1./("))
                best_fn = sp.lambdify("x0", expr=model.get_best()["equation"].replace("inv(", "1./("))
                
                tgt_fn = sp.lambdify("x0", expr=eqn)
            if "abs" in eqn:
                tgt_eqn = sp.simplify(eqn.replace("abs", ""))
                tgt_fn = sp.lambdify("x0", expr=eqn.replace("abs", ""))
            else:
                tgt_eqn = sp.simplify(eqn)
                tgt_fn = sp.lambdify("x0", expr=eqn)
        else:
            best_eqn = sp.simplify(model.get_best()["equation"].replace("inv(", "1./("))
            best_fn = sp.lambdify("x0", expr=model.get_best()["equation"].replace("inv(", "1./("))
            
            tgt_eqn = sp.simplify(eqn)
            tgt_fn = sp.lambdify("x0", expr=eqn)
            
        is_correct = sp.simplify(best_eqn - tgt_eqn) == 0
        my_mse = np.mean((tgt_fn(x) - best_fn(x))**2)
        
        equivalents.append(is_correct)
        mses.append(my_mse)
        
        wright = "correct" if equivalents[-1] else "incorrect"
        
        correct = 1 if equivalents[-1] else 0
        try:
            msg = f"eqn {hh+1}, trial {trial} predicted {wright} equation {best_eqn} for target {tgt_eqn}"
            msg += f" with mse {my_mse:.3}"
        except:
            msg = ""
        print(msg)
        
        #columns = "eqn, seed, pred, target, correct, mse, method, range_low, range_high, number_points"
        if logging:
            results = f"{eqn}, {hh}, {best_eqn}, {tgt_eqn}, {correct}, {my_mse},"\
                    f" {my_method}, {bottom}, {top}, {number_samples}"

            with open(f"{eval_tag}.csv", "a") as f:
                f.write(results)
                f.write("\n")

    msg = f"accuracy for equation {hh+1}: {np.mean(equivalents)}"\
            f" with mean mse: {np.mean(mses):3}"
    print(msg)
    accuracies.append(np.mean(equivalents))
    all_mses.append(np.mean(mses))
    all_mse_sds.append(np.std(mses))

In [ ]:
my_method = "PySR"
msg = f"{my_method} accuracies\n"

for ii, eqn in enumerate(benchmark_eqns):
    
    msg += f"\n  Complex-{ii+1},  accuracy: {accuracies[ii]:5f}, mse: {all_mses[ii]:.5} +/- {all_mse_sds[ii]:.5}\n"
    
    msg += f"  {sp.expand(eqn)} \n"

if logging:
    summary_file = f"{eval_tag}_summary.txt"
    with open(summary_file, "w") as f:
        f.write(msg)
print(msg)

In [ ]:
# needs to point to directory containing 
#  weights/100M.ckpt and jupyter/100M/eq_settings.json and jupyter/100M/config.yaml
my_method = "NSRTS"
eval_tag = f"eval_complex_{my_method}_{int(time.time())}"
columns = "eqn, seed, pred, target, correct, mse, method, range_low, range_high, number_points"

if logging:
    with open(f"{eval_tag}.csv", "w") as f:
        f.write(columns)
        f.write("\n")
    

nsrts_dir = "../../nsrts"

json_filepath = os.path.join(nsrts_dir, "jupyter", "100M", "eq_setting.json")
with open(json_filepath, 'r') as json_file:
    eq_setting = json.load(json_file)
     
cfg_filepath = os.path.join(nsrts_dir, "jupyter", "100M", "config.yaml")
cfg = omegaconf.OmegaConf.load(cfg_filepath)

weights_path = os.path.join(nsrts_dir, "weights", "100M.ckpt")
    
## Set up BFGS load rom the hydra config yaml
bfgs = BFGSParams(
        activated= cfg.inference.bfgs.activated,
        n_restarts=cfg.inference.bfgs.n_restarts,
        add_coefficients_if_not_existing=cfg.inference.bfgs.add_coefficients_if_not_existing,
        normalization_o=cfg.inference.bfgs.normalization_o,
        idx_remove=cfg.inference.bfgs.idx_remove,
        normalization_type=cfg.inference.bfgs.normalization_type,
        stop_time=cfg.inference.bfgs.stop_time,
    )

# adjust this parameter up for greater accuracy and longer runtime
cfg.inference.beam_size = 1

params_fit = FitParams(word2id=eq_setting["word2id"], 
                            id2word={int(k): v for k,v in eq_setting["id2word"].items()}, 
                            una_ops=eq_setting["una_ops"], 
                            bin_ops=eq_setting["bin_ops"], 
                            total_variables=list(eq_setting["total_variables"]),  
                            total_coefficients=list(eq_setting["total_coefficients"]),
                            rewrite_functions=list(eq_setting["rewrite_functions"]),
                            bfgs=bfgs,
                            beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            )

## Load equation configuration and architecture configuration
accuracies = []
all_mses = []
all_mse_sds = []

for hh, eqn in enumerate(benchmark_eqns):
    
    equivalents = []
    mses = []
    for trial in range(number_trials):

        np.random.seed(trial)
        torch.manual_seed(trial)

        
        ## Load architecture, set into eval mode, and pass the config parameters
        model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
        model.eval()
        if torch.cuda.is_available(): 
            model.to(torch.device("cuda:1")) 
            
        fitfunc = partial(model.fitfunc, cfg_params=params_fit)

        my_fn = sp.lambdify("x0", expr=eqn)
        
        
        print(x.shape, y.shape)
        # work-around for occasional catastrophic failure
        output = {"best_bfgs_preds": []}
        
        np.random.seed(trial)
        torch.manual_seed(trial)
        
        while len(output["best_bfgs_preds"]) == 0:
        
            (bottom, top, number_samples) = sample_meta[eqn]
            x = np.random.rand(number_samples, 1) \
                    * (top-bottom) \
                    + bottom
            y = my_fn(x)
            
            x = torch.tensor(x)
            y = torch.tensor(y)
            
            try:
                output = fitfunc(x, y.squeeze()) 
            except:
                output = fitfunc(x, y.squeeze()) 
            
        if (x >= 0).all():
            print("removing abs")
            # remove abs() if input only consists of positive numbers
            no_abs = output["best_bfgs_preds"][0].replace("Abs","")
            best_eqn = sp.simplify(no_abs.replace("x_1", "x0"))
            if "abs" in eqn:
                tgt_eqn = sp.simplify(eqn.replace("abs", ""))
            else:
                tgt_eqn = sp.simplify(eqn)
        else:
            best_eqn = sp.simplify(output["best_bfgs_preds"][0].replace("x_1", "x0"))
            tgt_eqn = sp.simplify(eqn)
        
        is_correct = sp.simplify(best_eqn - tgt_eqn) == 0
        my_mse = np.mean(((sp.lambdify("x0", expr=tgt_eqn)(x) - sp.lambdify("x0", expr=best_eqn)(x))**2).cpu().numpy())
        
        equivalents.append(is_correct)
        mses.append(my_mse)
        
        wright = "correct" if equivalents[-1] else "incorrect"
        correct = 1 if equivalents[-1] else 0
        
        msg = f"eqn {hh}, trial {trial} predicted {wright} equation {best_eqn} for target {tgt_eqn}"
        print(msg)
        
        #columns = "eqn, seed, pred, target, correct, mse, method, range_low, range_high, number_points"
        if logging:
            results = f"{eqn}, {hh}, {best_eqn}, {tgt_eqn}, {correct}, {my_mse},"\
                    f" {my_method}, {bottom}, {top}, {number_samples}"

            with open(f"{eval_tag}.csv", "a") as f:
                f.write(results)
                f.write("\n")

        
    msg = f"accuracy for equation {hh+1}: {np.mean(equivalents)}"\
            f" with mean mse: {np.mean(mses):3}"
    print(msg)
    accuracies.append(np.mean(equivalents))
    all_mses.append(np.mean(mses))
    all_mse_sds.append(np.std(mses))

In [ ]:
my_method = "NSRTS"
msg = f"{my_method} accuracies\n"

for ii, eqn in enumerate(benchmark_eqns):
    
    msg += f"\n  Complex-{ii+1},  accuracy: {accuracies[ii]:4f}, mse: {all_mses[ii]} +/- {all_mse_sds[ii]}\n"
    
    msg += f"  {sp.expand(eqn)} \n"

if logging:
    summary_file = f"{eval_tag}_summary.txt"
    with open(summary_file, "w") as f:
        f.write(msg)
print(msg)